In [104]:
import random
import pandas as pd

In [105]:
class customer:
    def __init__(self, NO):
        self.NO = NO 
        self.arrival = 0.0
        self.service_time = 0.0
        self.server_name = server('None', 2)
        self.end = 0.0
    def __eq__(self, other):
        return self.NO == other.NO
    def waiting_time(self):
        return
class server:
    def __init__(self, name, status = 0):
        self.name = name
        self.status = status #0 means idle and 1 means busy
    def __eq__(self, other):
        return self.name == other.name
def st(): # calculates the service time of a customer
    return random.uniform(9, 12)

def at(Tnow): # calculates the arrival time of a customer
    return random.uniform(3, 6) + Tnow

In [106]:
Able = server('Able', 0)
Baker = server('Baker', 0)

In [107]:
FEL = []
Queue = [] #list of inline customers
customers = [] #list of customers
Tnow = 0
Q = 0

st_range = (9,12) #range of service time (in min)
arr_range = (3,6) #range of arrival time (in min)
T = 9 * 60 #total time of simulation (in min)

In [108]:
def arrival(i):
    global FEL, Q, Tnow, customers, Queue
    # defining new customer
    c = customer(i)
    
    c.arrival = at(Tnow) #calculating new customer's arrival time
    arr_time = c.arrival
    customers.append(c) #adding new customer to the list of customer
    FEL.append([1, arr_time, c]) #adding new customer arrival event to FEL list
    FEL = sorted(FEL, key=lambda x: x[1])  # Sort based on arrival time
    #
    if Able.status == 0: #if able is free give customer to him
        Able.status = 1 
        customers[i - 1].service_time = st() 
        customers[i - 1].end = Tnow + customers[i - 1].service_time
        customers[i - 1].server_name = Able
        
        FEL.append([2, customers[i - 1].end, customers[i - 1]]) #adding data to FEL
        FEL = sorted(FEL, key=lambda x: x[1])  # Sort based on end time
        
    else:
        #assigning customer to baker
        if Baker.status == 0:
            Baker.status = 1
            customers[i - 1].service_time = st()
            customers[i - 1].end = Tnow + customers[i - 1].service_time
            customers[i - 1].server_name = Baker
            
            FEL.append([2, customers[i - 1].end, customers[i - 1]])
            FEL = sorted(FEL, key=lambda x: x[1])  # Sort based on end time
            
        else:
            Q = Q + 1
            Queue.append(customers[i - 1])


In [109]:
def departure(server, customer):
    global Q, FEL, Tnow, Queue
    
    if Q > 0:
        Q = Q - 1
        Queue[0].service_time = st()
        Queue[0].end = Tnow + Queue[0].service_time
        Queue[0].server_name = server
        FEL.append([2, Queue[0].end, Queue[0]])
        FEL = sorted(FEL, key=lambda x: x[1])  # Sort based on end time
        Queue.pop(0)
    else:
        server.status = 0
    
        

In [110]:
def waiting_line(wait):
    if abs(wait) < 0.1:
        result = 'No'
    else:
        result = 'Yes'
    return result

In [111]:
#Simulation Controller

c1 = customer(0)
c1.arrival = 0

customers.append(c1)
FEL.append([1,0,c1])
FEL = sorted(FEL, key=lambda x: x[1])  # Sort based on end time
i = 1 #number of customers
while True:
    temp = FEL.pop(0)
    Tnow = temp[1]
    if T > Tnow:
        
        if temp[0] == 1:
            arrival(i)
            i = i + 1
        elif temp[0] == 2:
            #departure    
            departure(temp[2].server_name, temp[2])
    else:
        break
            
        

In [112]:

final_table = []
header = ['customer No', 'clock of arrival', 'time service begins', 'service time', 'time service end', 'Server', 'waiting time', 'stayed in line?']
customers = sorted(customers, key=lambda x: x.NO)
for customer in customers:
    final_table.append([customer.NO + 1, customer.arrival, customer.end - customer.service_time, customer.service_time, customer.end, customer.server_name.name, customer.end - customer.service_time - customer.arrival, waiting_line(customer.end - customer.service_time - customer.arrival)])


## Results

In [113]:
df = pd.DataFrame(final_table)
df.columns = header
df.head(25)

,customer No,clock of arrival,time service begins,service time,time service end,Server,waiting time,stayed in line?
0,1,0.000000,0.000000,10.213671,10.213671,Able,0.000000e+00,No
1,2,3.648707,3.648707,11.442365,15.091072,Baker,-4.440892e-16,No
2,3,8.579197,10.213671,9.579604,19.793275,Able,1.634474e+00,Yes
3,4,11.618262,15.091072,10.535196,25.626268,Baker,3.472810e+00,Yes
4,5,17.061020,19.793275,10.038435,29.831710,Able,2.732255e+00,Yes
5,6,21.711276,25.626268,10.698455,36.324723,Baker,3.914992e+00,Yes
6,7,25.779505,29.831710,11.665752,41.497461,Able,4.052205e+00,Yes
7,8,29.414056,36.324723,11.899427,48.224150,Baker,6.910667e+00,Yes
8,9,33.231472,41.497461,9.873929,51.371390,Able,8.265989e+00,Yes
9,10,36.606519,48.224150,11.047700,59.271850,Baker,1.161763e+01,Yes


In [114]:
df.tail(25)

,customer No,clock of arrival,time service begins,service time,time service end,Server,waiting time,stayed in line?
98,99,438.382882,517.138367,10.365407,527.503774,Baker,78.755485,Yes
99,100,442.667609,519.695068,9.620742,529.315809,Able,77.027459,Yes
100,101,447.995199,527.503774,9.174067,536.677841,Baker,79.508575,Yes
101,102,453.527140,529.315809,10.571582,539.887391,Able,75.788669,Yes
102,103,458.848511,536.677841,11.061776,547.739617,Baker,77.829330,Yes
103,104,461.995838,539.887391,11.257890,551.145281,Able,77.891553,Yes
104,105,465.377225,0.000000,0.000000,0.000000,None,-465.377225,Yes
105,106,469.834658,0.000000,0.000000,0.000000,None,-469.834658,Yes
106,107,475.339326,0.000000,0.000000,0.000000,None,-475.339326,Yes
107,108,479.246795,0.000000,0.000000,0.000000,None,-479.246795,Yes


In [115]:
#calculating average waiting time
total_waiting_time = 0
stayed_inline = 0
served_customers = 0
for customer in customers:
    if customer.server_name.name != 'None':
            served_customers += 1
            total_waiting_time += customer.end - customer.service_time - customer.arrival
    if waiting_line(customer.end - customer.service_time - customer.arrival) == 'Yes':
        stayed_inline += 1
        
average_waiting_time = total_waiting_time / served_customers
print(f'The average waiting time is: {average_waiting_time}\nTotal customers stayed in line: {stayed_inline}')
served_customers

The average waiting time is: 42.92958281899132
Total customers stayed in line: 121


104

In [116]:
Able_utilization = 0
Baker_utilization = 0
for customer in customers:
    if customer.server_name == Able:
        Able_utilization += customer.service_time
    else:
        Baker_utilization += customer.service_time
Total_time = customers[served_customers - 1].end
Baker_utilization = Baker_utilization / Total_time * 100
Able_utilization = Able_utilization /Total_time * 100
print(f'Bakers utilization percent is: {Baker_utilization}\nAbles utilization percent is: {Able_utilization}')

Bakers utilization percent is: 98.720052384623
Ables utilization percent is: 100.0
